# Feature Pipeline using Synthetic Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/featurestoreorg/serverless-ml-course/blob/main/src/02-module/2_cc_feature_pipeline.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

## 🗒️ This notebook is divided in 2 sections:
1. Reading the synthetic credit card data and feature engineeing,
2. Write the Pandas DataFrames to the feature groups in the feature store.


In [1]:
#!pip install -U hopsworks --quiet
!pip install -U faker --quiet

In [2]:
import pandas as pd
import datetime
import hopsworks
from sml import synthetic_data
import random
pd.options.mode.chained_assignment = None

/home/codespace/miniconda3/envs/mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
start_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
print(start_time)

2024-03-21 12:30:35


In [4]:
#end_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(end_time)

2024-03-22 12:30:45


In [5]:
synthetic_data.FRAUD_RATIO = random.uniform(0.001, 0.005)
synthetic_data.TOTAL_UNIQUE_USERS = 1000
synthetic_data.TOTAL_UNIQUE_TRANSACTIONS = 54000
synthetic_data.CASH_WITHRAWAL_CARDS_TOTAL = 2000
synthetic_data.TOTAL_UNIQUE_CASH_WITHDRAWALS = 200
synthetic_data.START_DATE=start_time
synthetic_data.END_DATE=end_time

credit_cards = synthetic_data.generate_list_credit_card_numbers()
credit_cards_df = synthetic_data.create_credit_cards_as_df(credit_cards)
profiles_df = synthetic_data.create_profiles_as_df(credit_cards)
trans_df = synthetic_data.create_transactions_as_df(credit_cards)

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning you will create additional features based on these patterns. In particular, you will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

In [6]:
fraud_labels = trans_df.copy()[["tid", "cc_num", "datetime", "fraud_label"]]
fraud_labels

,tid,cc_num,datetime,fraud_label
0,b564311a93240aa10d813abab11e0270,4263720412621721,2024-03-21 12:30:38,0
1,7b0b89f5d9a21927167b68419bdae21a,4975544600922082,2024-03-21 12:30:39,0
2,52713410a9c36387bab9a5e20f1b896a,4027085330355977,2024-03-21 12:30:40,0
3,ba0e4f47895c7c76ccae0adb14f6b697,4343236168322947,2024-03-21 12:30:42,0
4,77e9975c70fa2c25af71554c4b0487d0,4581222106011785,2024-03-21 12:30:45,0
...,...,...,...,...
60081,3ee4c9882709e836717b87dcd499863f,4179358160776166,2024-04-09 00:42:46,0
60082,5622b5d1a1f4eced2a1698c2f24f8193,4179358160776166,2024-04-12 02:42:46,0
60083,dc90c63128fdbfbf7130acf5b598b7f1,4179358160776166,2024-04-15 04:42:46,0
60084,c09f748d961e9f430ffe905d5f6b58c8,4179358160776166,2024-04-18 06:42:46,0


In [7]:
from sml import cc_features

fraud_labels.datetime = fraud_labels.datetime.map(lambda x: cc_features.date_to_timestamp(x))
fraud_labels

,tid,cc_num,datetime,fraud_label
0,b564311a93240aa10d813abab11e0270,4263720412621721,1711024238000,0
1,7b0b89f5d9a21927167b68419bdae21a,4975544600922082,1711024239000,0
2,52713410a9c36387bab9a5e20f1b896a,4027085330355977,1711024240000,0
3,ba0e4f47895c7c76ccae0adb14f6b697,4343236168322947,1711024242000,0
4,77e9975c70fa2c25af71554c4b0487d0,4581222106011785,1711024245000,0
...,...,...,...,...
60081,3ee4c9882709e836717b87dcd499863f,4179358160776166,1712623366000,0
60082,5622b5d1a1f4eced2a1698c2f24f8193,4179358160776166,1712889766000,0
60083,dc90c63128fdbfbf7130acf5b598b7f1,4179358160776166,1713156166000,0
60084,c09f748d961e9f430ffe905d5f6b58c8,4179358160776166,1713422566000,0


In [8]:
trans_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,b564311a93240aa10d813abab11e0270,2024-03-21 12:30:38,4263720412621721,Electronics,456.97,36.025060,-86.779170,Brentwood Estates,US,0
1,7b0b89f5d9a21927167b68419bdae21a,2024-03-21 12:30:39,4975544600922082,Grocery,71.93,33.410120,-91.061770,Greenville,US,0
2,52713410a9c36387bab9a5e20f1b896a,2024-03-21 12:30:40,4027085330355977,Restaurant/Cafeteria,10.87,33.036990,-117.291980,Encinitas,US,0
3,ba0e4f47895c7c76ccae0adb14f6b697,2024-03-21 12:30:42,4343236168322947,Grocery,79.64,29.845760,-90.106740,Estelle,US,0
4,77e9975c70fa2c25af71554c4b0487d0,2024-03-21 12:30:45,4581222106011785,Holliday/Travel,43.42,40.605380,-73.755130,Far Rockaway,US,0
...,...,...,...,...,...,...,...,...,...,...
60081,3ee4c9882709e836717b87dcd499863f,2024-04-09 00:42:46,4179358160776166,Cash Withdrawal,62.36,34.239010,-119.044274,Camarillo,US,0
60082,5622b5d1a1f4eced2a1698c2f24f8193,2024-04-12 02:42:46,4179358160776166,Cash Withdrawal,79.94,34.248261,-119.041913,Camarillo,US,0
60083,dc90c63128fdbfbf7130acf5b598b7f1,2024-04-15 04:42:46,4179358160776166,Cash Withdrawal,2.69,34.243697,-119.033705,Camarillo,US,0
60084,c09f748d961e9f430ffe905d5f6b58c8,2024-04-18 06:42:46,4179358160776166,Cash Withdrawal,16.80,34.234965,-119.040798,Camarillo,US,0


In [9]:
trans_df.drop(['fraud_label'], inplace = True, axis=1)

In [10]:
trans_df = cc_features.card_owner_age(trans_df, profiles_df)
trans_df = cc_features.expiry_days(trans_df, credit_cards_df)
trans_df = cc_features.activity_level(trans_df, 1)

In [11]:
window_len = 4
window_aggs_df = cc_features.aggregate_activity_by_hour(trans_df, window_len)

Next, you will create features that for each credit card aggregate data from multiple time steps.

Yoy will start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

Next lets compute windowed aggregates. Here you will use 4-hour windows, but feel free to experiment with different window lengths by setting `window_len` below to a value of your choice.

In [12]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/556180
Connected. Call `.close()` to terminate connection gracefully.


To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [14]:
trans_fg = fs.get_feature_group(name="cc_trans_fraud", version=2)
trans_fg.insert(trans_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: cc_trans_fraud_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/556180/jobs/named/cc_trans_fraud_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fbc74b12410>, None)

In [15]:
window_aggs_fg = fs.get_feature_group(name=f"cc_trans_fraud_{window_len}h", version=2)
window_aggs_fg.insert(window_aggs_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: cc_trans_fraud_4h_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/556180/jobs/named/cc_trans_fraud_4h_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fbc754c6390>, None)

In [16]:

labels_fg = fs.get_feature_group(name="transactions_fraud_label", version=2)
labels_fg.insert(fraud_labels)

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: transactions_fraud_label_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/556180/jobs/named/transactions_fraud_label_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fbc74b33750>, None)